In [31]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [32]:
X = cv2.imread('./images/simple_binary.png')

In [33]:
X = cv2.cvtColor(X,cv2.COLOR_BGR2GRAY)
X = X[0:234][0:234]

In [39]:
p_size = 80
patches_x = 3
patches_y = 3

In [43]:
offset = 1
for p_x in range(patches_x-1):
    p = 0
    for p_y in range(patches_y-1):
        cv2.imwrite(f'image_{offset}.png', X[p:p_x*p_size][p:     p_y*p_size])
        print(f'image{offset} done!\n')
        offset += 1

image1 done!

image2 done!

image3 done!

image4 done!

image5 done!

image6 done!



IndexError: index 234 is out of bounds for axis 0 with size 234

True